This helper makes sure that the packages that rebuilt successfully with Python 3.13 but fail to install due to their dependencies not yet rebuilt have bugzillas blocked by the appropriate bugzillas of said dependencies.

 1. For every package that has been successfully rebuilt, we see if it has a bugzilla that blocks `PYTHON3.13` and `F41FailsToInstall`.
 2. If so, we parse the initial comment for missing `python3.13dist()` dependencies.
 3. We convert the dependencies to `python3.12dist()` dependencies and find their component.
 4. If the component has a `PYTHON3.13` bugzilla, we mark it as blocking the bugzilla from (1).

In [11]:
from IPython.display import display, Markdown

In [12]:
import bugzilla
bzapi = bugzilla.Bugzilla('bugzilla.redhat.com')
query = bzapi.build_query(product='Fedora')
query['blocks'] = 2244836  # PYTHON3.13
query['limit'] = 20 # Bugzilla page size
query['offset'] = 0
query['status'] = '__open__'
bugz = []
while len(partial := bzapi.query(query)) == 20:
    bugz += partial
    query['offset'] += 20
    print(len(bugz))
bugz += partial

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800


In [13]:
F41FailsToInstall = 2260877

In [14]:
python313 = !cat python313.pkgs

In [15]:
len(pure_fti_bugz := [b for b in bugz if b.component in python313 and F41FailsToInstall in b.blocks])
pure_fti_bugz

[<Bug #2291488 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78f0075700>,
 <Bug #2291501 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78ebf1ccb0>,
 <Bug #2291502 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78f00e7860>,
 <Bug #2291514 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78f00e7500>,
 <Bug #2291525 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78ebf2e4b0>,
 <Bug #2291545 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78ebf2e3f0>,
 <Bug #2291568 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78f0017c80>,
 <Bug #2291574 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78f00281a0>,
 <Bug #2291581 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78de7db260>,
 <Bug #2291582 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78f00e7fe0>,
 <Bug #2291583 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78f00e7ef0>,
 <Bug #2291601 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78f008dc10>,
 <Bug #2291602 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f78de7daff0>,

In [17]:
for bug in pure_fti_bugz:
    display(Markdown(f'# {bug.component}\n[rbz#{bug.id}](https://bugzilla.redhat.com/{bug.id})'))
    comments = bug.getcomments()
    initial_comment = comments[0]
    if initial_comment['creator'] != 'fti-bugs@fedoraproject.org':
        print('Not an automated FTI bug by Miro')
        continue

    blocked_by = set()
    lines = initial_comment['text'].splitlines()
    for line in lines:
        if line.startswith('  - nothing provides '):
            print(line)
            provide = line.split(' ')[5]
            if provide.startswith('('):
                provide = provide[1:]
            if provide.startswith('python3.13dist('):
                old_provide = provide.replace('python3.13dist(', 'python3.12dist(')
                blocker_components = !repoquery -q --repo=rawhide --source --whatprovides "{old_provide}" | pkgname
                if len(set(blocker_components)) == 1:
                    blocker_component = blocker_components[0]
                    blocker_bugz = [b for b in bugz if b.component == blocker_component]
                    if len(blocker_bugz) > 1:
                        print(f"    = too many bugzillas for {blocker_component}")
                        for bug in blocker_bugz:
                            print(f"        https://bugzilla.redhat.com/{bug.id}")
                    elif blocker_bugz:
                        bid = blocker_bugz[0].id
                        print(f"    = blocked by {blocker_component} https://bugzilla.redhat.com/{bid}")
                        blocked_by.add(bid)
                else:
                    print('Check:', blocker_components)
    print(blocked_by)
    #if blocked_by:
    #    add_blockers = bzapi.build_update(depends_on_add=sorted(blocked_by))
    #    bzapi.update_bugs([bug.id], add_blockers)

# atomic-reactor
[rbz#2291488](https://bugzilla.redhat.com/2291488)

  - nothing provides python3.13dist(flatpak-module-tools) >= 0.11 needed by python3-atomic-reactor-3.14.0-8.fc41.noarch
    = too many bugzillas for flatpak-module-tools
        https://bugzilla.redhat.com/2252072
        https://bugzilla.redhat.com/2291522
set()


# colin
[rbz#2291501](https://bugzilla.redhat.com/2291501)

  - nothing provides python3.13dist(fmf) needed by python3-colin-0.5.3-9.fc41.noarch
    = too many bugzillas for fmf
        https://bugzilla.redhat.com/2279994
        https://bugzilla.redhat.com/2291523
set()


# conda
[rbz#2291502](https://bugzilla.redhat.com/2291502)

...

  - nothing provides python3.13dist(multipart) needed by trac-1.6-3.fc41.noarch
Check: []
set()


# vdirsyncer
[rbz#2292017](https://bugzilla.redhat.com/2292017)

  - nothing provides (python3.13dist(aiostream) < 0.6~~ with python3.13dist(aiostream) >= 0.4.3) needed by python3-vdirsyncer-0.19.2-3.fc41.noarch
    = blocked by python-aiostream https://bugzilla.redhat.com/2291611
{2291611}


# zanata-python-client
[rbz#2292023](https://bugzilla.redhat.com/2292023)

  - nothing provides python3.13dist(future) needed by python3-zanata-client-1.5.3-19.fc41.noarch
    = too many bugzillas for future
        https://bugzilla.redhat.com/2250662
        https://bugzilla.redhat.com/2291526
set()
